### Step 1: Imports

In [ ]:
import torch
import torch.nn as nn

import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
import seaborn as sns

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # TODO get GPU

from src.utils.get_data import get_data_loader
from src.utils.get_data import import_data

### Step 2: Get Data

In [ ]:
DATA_PATH = "../data"
fc, lc = import_data(DATA_PATH, segmentation_type='coarse', is_user_features=True,
                     return_type='pd')
fn, ln = import_data(DATA_PATH, segmentation_type='no', is_user_features=True,
                     return_type='pd')

### Step 2.1: Data Analysis

#### Class Imbalance: We observe quite the heavy imbalance, in favor of label 0

In [ ]:
sns.countplot(x = 'Label', data=ln)
print('Original dataset shape %s' % Counter(ln["Label"]))
print(fn.shape, ln.shape)

In [ ]:
oversample = SMOTE(random_state=42)
fn_oversampled, ln_oversampled = oversample.fit_resample(fn, ln)

fn_oversampled = pd.DataFrame(fn_oversampled, columns=fn.columns)
ln_oversampled = pd.DataFrame(ln_oversampled, columns=ln.columns)

sns.countplot(x = 'Label', data=ln_oversampled)
print('Resampled dataset shape %s' % Counter(ln_oversampled["Label"]))

##### Split into Training and Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fn, ln["Label"], test_size=0.33, random_state=420)

#### Standardize Inputs

In [ ]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

### Step 3: Hyper Parameters

In [ ]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001
INPUT_DIM = X_train.shape[1]
OUTPUT_DIM = 1
HIDDEN1_DIM = 100
HIDDEN2_DIM = 10
DROPOUT = 0.1
WEIGHT_DECAY = 1

### Step 4: Define Custom Data Loaders

In [ ]:
## train data
"""
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
test_data = trainData(torch.FloatTensor(X_test), torch.FloatTensor(y_test.values))
"""

In [ ]:
#train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
#test_loader = DataLoader(dataset=test_data, batch_size=1)

In [ ]:
train_loader, test_loader = get_data_loader("no", smote=True, batch_size=64)

### Step 5: Define Custom NN

In [ ]:
class BinaryClassification(nn.Module):
    def __init__(self, input_dim, output_dim, hidden1_dim, hidden2_dim, dropout=0.1):
        super(BinaryClassification, self).__init__()        # Number of input features is 12.
        self.layer_1 = nn.Linear(input_dim, hidden1_dim) 
        self.layer_2 = nn.Linear(hidden1_dim, hidden2_dim)
        self.layer_out = nn.Linear(hidden2_dim, output_dim) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
        self.batchnorm1 = nn.BatchNorm1d(hidden1_dim)
        self.batchnorm2 = nn.BatchNorm1d(hidden2_dim)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [ ]:
### Step 6: TODO

In [ ]:
model = BinaryClassification(INPUT_DIM, OUTPUT_DIM, HIDDEN1_DIM, HIDDEN2_DIM, DROPOUT)
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

from sklearn import metrics

def aoc(y_pred, y_test):
    y_pred = y_pred.detach().numpy()
    y_test = y_test.detach().numpy()
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    return metrics.auc(fpr, tpr)

### Step X: Train the Model

In [ ]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        #acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        acc = aoc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')
    #print(acc1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
epoch_acc

### Step X: Test the Model 

In [ ]:
y_pred_list = []
y_test_list = []
model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader: # todo for now i onyl include labels from data loader as _ THIS IS BAD! ITS WRONG!
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_test_list.append(y_batch.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
y_test_list = [b.squeeze().tolist() for b in y_test_list]

In [ ]:
len(y_test_list)

In [ ]:
c = confusion_matrix(y_test_list, y_pred_list)

In [ ]:
print(classification_report(y_test_list, y_pred_list))

# TODO

first off, you wouldn't shuffle your test loader.

Here's an example:
 
batch_size = 20  
class_sample_count = [10, 1, 20, 3, 4] # dataset has 10 class-1 samples, 1 class-2 samples, etc.  
weights = 1 / torch.Tensor(class_sample_count)  
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, batch_size)  
trainloader = data_utils.DataLoader(train_dataset, batch_size = batch_size, shuffle=True, sampler = sampler) 

I THINK THIS OR SMOTE: BASICALLY THIS IS DEFAULT OVERSAMPLING
